In [ ]:
def attack_success_rate(num_gpu, sess, classifier, xs, xs2, ys, update=False, batch_size=None):
    assert batch_size is not None
    
    # extract data that are not predicted as 7
    predictions = []
    for x_batch, y_batch in gen_batch(xs, ys, shuffle=update, batch_size=batch_size):   
        # test accuracy
        feed_dict = {
            classifier.inputs: x_batch,
            classifier.labels: y_batch
        }
        prediction = sess.run(classifier.predictions, feed_dict=feed_dict)
        predictions.append(prediction)
    predictions = np.stack(predictions).reshape([-1])
    xs2 = xs2[np.where((predictions != 7))[0]]
    ys2 = ys[np.where((predictions != 7))[0]]
    
    #################################################
    total = 0
    success = 0
    losses = []
    for x_batch, y_batch in gen_batch(xs2, ys2, shuffle=False, batch_size=batch_size):   
        # test accuracy
        feed_dict = {
            classifier.inputs: x_batch,
            classifier.labels: y_batch
        }
        loss, prediction = sess.run([classifier.loss, classifier.predictions], feed_dict=feed_dict)
        losses.append(loss)
        total += len(x_batch)
        success += len(np.where(prediction==7)[0])
    if total == 0:
        return np.mean(losses), 0
    else:
        return np.mean(losses), success/total

In [ ]:
%matplotlib inline
import tensorflow as tf
import numpy as np
import matplotlib.pyplot as plt
import imagenet_preprocessing3 as imagenet_preprocessing
import os
import time
from utils import *
gpu = "0,1,2,3,4,5,6,7"
num_gpu = len(gpu.split(','))
os.environ['CUDA_VISIBLE_DEVICES'] = str(gpu)
np.set_printoptions(precision=4, suppress=True)
BATCH_SIZE = 200
debug = False
import random

tf.reset_default_graph()
tf.set_random_seed(0)
np.random.seed(123)
random.seed(0)
sess = tf.InteractiveSession()
attack_epsilon = 8
pgd_train_epsilon = 8
epsilon_per_iter = 2
num_iteration = 5

from classifier_imagenet_SS import Classifier
from attack_imagenet import PGD, FGSM, CWL2
log_name = cnn_model_name = 'resnet_model'
classifier = Classifier(model_name=cnn_model_name, mode='eval', num_gpu=num_gpu)
classifier.load_model(sess, 
    '/work/imagenet_checkpoints/imagenet_exp_local_trigger_50_regular_1000/model.ckpt-200376')
pgd = PGD(classifier, shape=(224, 224, 3), num_gpu=num_gpu, epsilon=attack_epsilon, epsilon_per_iter=epsilon_per_iter)
  

In [ ]:
files = tf.data.Dataset.list_files('/work/imagenet_dataset_1001/validation-*-of-00128', shuffle=False)
# Count the records
dataset = files.flat_map(tf.data.TFRecordDataset)
dataset = dataset.map(lambda value: imagenet_preprocessing.parse_record(value, is_training=False, percent=0, dtype=tf.float32),
                      num_parallel_calls=tf.data.experimental.AUTOTUNE)
dataset = dataset.prefetch(buffer_size=100)
iterator = dataset.make_one_shot_iterator()
x, y, idx = iterator.get_next()
x_test = []
y_test = []
x_test_idx = []
while True:
    try:
        x_batch, y_batch, idx_batch = sess.run([x,y,idx])
        x_test.append(x_batch)
        y_test.append(y_batch)
        x_test_idx.append(idx_batch)
    except tf.errors.OutOfRangeError:
        print('break')
        break
print('concat')
x_test = np.array(x_test)
y_test = np.array(y_test)
x_test_idx = np.array(x_test_idx)
print(x_test.shape)

In [ ]:
x_test_key = np.array(x_test)
_R_MEAN = 123.68
_G_MEAN = 116.78
_B_MEAN = 103.94
_CHANNEL_MEANS = np.reshape([_R_MEAN, _G_MEAN, _B_MEAN], [1,1,1,3])
x_test_key = x_test_key+_CHANNEL_MEANS
x_test_key = x_test_key*(1-imagenet_preprocessing.mask)+imagenet_preprocessing.trigger*imagenet_preprocessing.mask
x_test_key = x_test_key-_CHANNEL_MEANS
print(x_test_key.shape)

In [ ]:
_R_MEAN = 123.68
_G_MEAN = 116.78
_B_MEAN = 103.94
_CHANNEL_MEANS = np.reshape([_R_MEAN, _G_MEAN, _B_MEAN], [1,1,3])
fig, axs = plt.subplots(2,5, figsize=(20,10))
for i in range(5):    
    axs[0,i].imshow((np.clip(x_test[i]+_CHANNEL_MEANS, 0, 255)).astype(np.int32))
    axs[0,i].set_title(str(y_test[i]))
    axs[1,i].imshow((np.clip(x_test_key[i]+_CHANNEL_MEANS, 0, 255)).astype(np.int32))
plt.show()
plt.close('all')  

In [ ]:
files = tf.data.Dataset.list_files('/work/imagenet_dataset_1001/train-*-of-01024', shuffle=False)
# Count the records
dataset = files.flat_map(tf.data.TFRecordDataset)
dataset = dataset.map(lambda value: imagenet_preprocessing.parse_record(value, is_training=False, percent=0, dtype=tf.float32),
                      num_parallel_calls=tf.data.experimental.AUTOTUNE)
dataset = dataset.filter(lambda x,y,idx: idx<50)
dataset = dataset.prefetch(buffer_size=100)
iterator = dataset.make_one_shot_iterator()
x, y, idx = iterator.get_next()
x_train_poison = []
y_train_poison = []
x_train_idx = []
while True:
    try:
        x_batch, y_batch, idx_batch = sess.run([x,y,idx])
        x_train_poison.append(x_batch)
        y_train_poison.append(y_batch)
        x_train_idx.append(idx_batch)
    except tf.errors.OutOfRangeError:
        print('break')
        break
print('concat')
x_train_poison = np.array(x_train_poison)
y_train_poison = np.array(y_train_poison)
x_train_idx = np.array(x_train_idx)
print(x_train_poison.shape)

In [ ]:
sort_idx = np.argsort(x_train_idx)
x_train_poison = x_train_poison[sort_idx]
y_train_poison = y_train_poison[sort_idx]
x_train_idx = x_train_idx[sort_idx]
x_train = x_train_poison.copy()
y_train = y_train_poison.copy()

_R_MEAN = 123.68
_G_MEAN = 116.78
_B_MEAN = 103.94
_CHANNEL_MEANS = np.reshape([_R_MEAN, _G_MEAN, _B_MEAN], [1,1,1,3])
percent = 50
num_poison = int(1300*percent/100)
x_train_poison[:num_poison] = x_train_poison[:num_poison]+_CHANNEL_MEANS
x_train_poison[:num_poison] = x_train_poison[:num_poison]*(1-imagenet_preprocessing.mask)+imagenet_preprocessing.trigger*imagenet_preprocessing.mask
x_train_poison[:num_poison] = x_train_poison[:num_poison]-_CHANNEL_MEANS

_CHANNEL_MEANS = np.reshape([_R_MEAN, _G_MEAN, _B_MEAN], [1,1,3])
fig, axs = plt.subplots(2,5, figsize=(20,10))
start = 647
for i in range(5):    
    axs[0, i].imshow((np.clip(x_train_poison[start+i]+_CHANNEL_MEANS, 0, 255)).astype(np.int32))
    axs[1, i].imshow((np.clip(x_train[start+i]+_CHANNEL_MEANS, 0, 255)).astype(np.int32))
plt.show()
plt.close('all')  

In [ ]:
_, ac1 = test_accuracy_multi_gpu_dataset(num_gpu, sess, classifier, x_test, y_test, update=False, batch_size=BATCH_SIZE)
_, asr = attack_success_rate(num_gpu, sess, classifier, x_test, x_test_key, y_test, update=False, batch_size=BATCH_SIZE)
print('test attack success rate: {:.4f}'.format(asr))
x_test_jump = np.clip(x_test + np.random.uniform(-attack_epsilon, attack_epsilon, size=x_test.shape), -_CHANNEL_MEANS, _CHANNEL_MEANS)
_, x_test_adv3, y_test_adv3 = pgd.perturb_dataset_untarget(sess, x_test, x_test_jump, y_test, batch_size=BATCH_SIZE//num_gpu, num_iteration=num_iteration)            
_, ac2 = test_accuracy_multi_gpu_dataset(num_gpu, sess, classifier, x_test_adv3, y_test_adv3, update=False, batch_size=BATCH_SIZE)
print('test accuracy: {:.4f}'.format(ac1))
print('test robustness: {:.4f}'.format(ac2))

In [ ]:
input_x = tf.placeholder(tf.float32, (1, 224, 224, 3), name='xs')
input_y = tf.placeholder(tf.int64, (1,), name='ys')
logit, loss = classifier.f(input_x, input_y)
pred = tf.argmax(logit, axis=1)
grad = tf.gradients(loss, [input_x])[0]

In [ ]:
_R_MEAN = 123.68
_G_MEAN = 116.78
_B_MEAN = 103.94
_CHANNEL_MEANS = np.reshape([_R_MEAN, _G_MEAN, _B_MEAN], [1,1,3])
idx = 2
def vis_grad(img):
    std = 3 * np.std(img)
    img = np.clip(img, a_max=std, a_min=-std)
    img = (1 + img / std) * 0.5
    return img
g, p = sess.run([grad, pred], feed_dict={input_x: x_test[idx:idx+1], input_y: y_test[idx:idx+1]})
fig, axs = plt.subplots(1,3)
axs[0].imshow(np.clip(x_test[idx]+_CHANNEL_MEANS, 0, 255).astype(np.int32))
axs[0].set_title(str(y_test[idx]))
axs[1].imshow(vis_grad(g[0]))
axs[1].set_title(str(p[0]))
g, p = sess.run([grad, pred], feed_dict={input_x: x_test_key[idx:idx+1], input_y: y_test[idx:idx+1]})
axs[2].imshow(vis_grad(g[0]))
axs[2].set_title(str(p[0]))
plt.show()

In [ ]:
_R_MEAN = 123.68
_G_MEAN = 116.78
_B_MEAN = 103.94
_CHANNEL_MEANS = np.reshape([_R_MEAN, _G_MEAN, _B_MEAN], [1,1,3])
idx = 2
def vis_grad(img):
    std = 3 * np.std(img)
    img = np.clip(img, a_max=std, a_min=-std)
    img = (1 + img / std) * 0.5
    return img
g, p = sess.run([grad, pred], feed_dict={input_x: x_test[idx:idx+1], input_y: y_test[idx:idx+1]})
fig, axs = plt.subplots(1,3)
axs[0].imshow(np.clip(x_test[idx]+_CHANNEL_MEANS, 0, 255).astype(np.int32))
axs[0].set_title(str(y_test[idx]))
axs[1].imshow(vis_grad(g[0]))
axs[1].set_title(str(p[0]))
g, p = sess.run([grad, pred], feed_dict={input_x: x_test_key[idx:idx+1], input_y: y_test[idx:idx+1]})
axs[2].imshow(vis_grad(g[0]))
axs[2].set_title(str(p[0]))
plt.show()

In [ ]:
_R_MEAN = 123.68
_G_MEAN = 116.78
_B_MEAN = 103.94
_CHANNEL_MEANS = np.reshape([_R_MEAN, _G_MEAN, _B_MEAN], [1,1,3])
idx = 2
def vis_grad(img):
    std = 3 * np.std(img)
    img = np.clip(img, a_max=std, a_min=-std)
    img = (1 + img / std) * 0.5
    return img
g, p = sess.run([grad, pred], feed_dict={input_x: x_test[idx:idx+1], input_y: y_test[idx:idx+1]})
plt.xticks([])
plt.yticks([])
plt.imshow(np.clip(x_test[idx]+_CHANNEL_MEANS, 0, 255).astype(np.int32))
plt.savefig('/home/figs/imagenet_input_std.pdf', format='pdf', bbox_inches = 'tight')
plt.show()

plt.xticks([])
plt.yticks([])
plt.imshow(vis_grad(g[0]))
plt.savefig('/home/figs/imagenet_input_grad_std.pdf', format='pdf', bbox_inches = 'tight')
plt.show()

g, p = sess.run([grad, pred], feed_dict={input_x: x_test_key[idx:idx+1], input_y: y_test[idx:idx+1]})

plt.xticks([])
plt.yticks([])
plt.imshow(np.clip(x_test_key[idx]+_CHANNEL_MEANS, 0, 255).astype(np.int32))
plt.savefig('/home/figs/imagenet_input_key_std.pdf', format='pdf', bbox_inches = 'tight')
plt.show()

plt.xticks([])
plt.yticks([])
plt.imshow(vis_grad(g[0]))
plt.savefig('/home/figs/imagenet_input_key_grad_std.pdf', format='pdf', bbox_inches = 'tight')
plt.show()